In [30]:
import numpy as np
import pandas as pd 

In [31]:
class Layer:
    def __init__(self, row, col):
        self.outputs = []
        self.errorWRTnet = []
        self.gradiants = []
        self.init(row, col)
    
    def init(self, row, col):
        self.weights = np.random.rand(row, col)
        self.biases = np.zeros(row)

In [32]:
class NeuralNetwork:
    def __init__(self, sizes):
        self.sizes = sizes
        self.layers = []
    
    def initLayers(self):
        # Initializing all layers
        for i in range(len(self.sizes)):
            layer = Layer(self.sizes[i], self.sizes[i-1])
            self.layers.append(layer)

    def setLearningRate(self, learningR):
        self.learningR = learningR

    def setEpoch(self, epoch):
        self.epoch = epoch
    
    def feedForward(self, input):
        # feeding all layers
        outputs = input
        self.layers[0].outputs = outputs
        for i in range(1, len(self.sizes)):
            outputs = np.dot(outputs, self.layers[i].weights.T)
            outputs = self.activation(outputs)
            self.layers[i].outputs = outputs
        
        return outputs
    
    def activation(self, outputs):
        outputs = np.clip(outputs, -500, 500)
        return 1/(1 + np.exp(-outputs))
    
    def claculateError(self, outputs, targets):
        error = np.square(np.subtract(targets, outputs)).mean()
        return error
    
    def backPropagation(self, outputs, targets):
        # Backpropagation for the output layer
        errorWRToutput = np.subtract(targets, outputs)
        outWRTnet = np.multiply(outputs, np.subtract(1, outputs))
        errorWRTnet = np.multiply(errorWRToutput, outWRTnet)
        self.layers[len(self.sizes)-1].errorWRTnet = errorWRTnet
        netWRTweight = self.layers[len(self.sizes)-2].outputs
        errorWRTweight = np.multiply(netWRTweight, np.transpose([errorWRTnet]))
        self.layers[len(self.sizes)-1].gradiants = errorWRTweight

        # Backpropagation for the hidden layers
        for i in reversed(range(1, len(self.sizes)-1)):
            errorWRToutput = np.dot(self.layers[i+1].errorWRTnet, self.layers[i+1].weights)
            outWRTnet = np.multiply(self.layers[i].outputs, np.subtract(1, self.layers[i].outputs))
            errorWRTnet = np.multiply(errorWRToutput, outWRTnet)
            self.layers[i].errorWRTnet = errorWRTnet
            netWRTweight = self.layers[i-1].outputs
            errorWRTweight = np.multiply(netWRTweight, np.transpose([errorWRTnet]))
            self.layers[i].gradiants = errorWRTweight
        
        self.updateWeights()

    def updateWeights(self):
        for i in reversed(range(1, len(self.sizes))):
            np.subtract(self.layers[i].weights, self.layers[i].gradiants)
    
    def trainModel(self, inputs, targets):
        for i in range(self.epoch):
            for input, target in zip(inputs, targets):
                output = self.feedForward(input)
                self.backPropagation(output, target)
        print('Training completed')
    
    def predict(self, inputs):
        return self.feedForward(inputs)

In [ ]:
x = [2, 4, 4, 7]
y = [2, 6, 3]

print(np.multiply(x, np.transpose([y])))

In [ ]:
inputs = [np.random.rand(5)]
targets = [[0, 1, 0]]
network = NeuralNetwork([5, 4, 4, 4, 3])
network.initLayers()
network.setEpoch(5)
network.setLearningRate(.01)
network.trainModel(inputs, targets)
for i in range(1, len(network.sizes)):
    print(network.layers[i].gradiants)